<a href="https://colab.research.google.com/github/mahendra-gehlot/CP260/blob/main/Vision_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2 as cv
import glob 
import matplotlib.pyplot as plt

SKIP_STEP = 20

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
O_PATH = "/content/drive/MyDrive/Vision/Given_Outdoor_data/frame_[0-403]*.png"
I_PATH = "/content/drive/MyDrive/Vision/Given_Indoor_data/frame[0-290]*.png"

In [ ]:
images = []
for img in glob.glob(O_PATH):
    images.append(img)
    
print(len(images))

404


In [ ]:
# Sampling image after every 20th frame
imgs = []
for i in range(0,len(images),SKIP_STEP):
    I = cv.imread(images[i],0)
    imgs.append(I)

In [ ]:
def extract_feature_SIFT(image):

    sift = cv.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(image,None)

    
    return kp, des

In [ ]:
def extract_feature_SURF(image):

    surf = cv.xfeatures2d.SURF_create(400)
    kp, des = surf.detectAndCompute(image,None)

    return kp, des

In [ ]:
def extract_feature_ORB(image):

    orb = cv.ORB_create()
    kp = orb.detect(image,None)
    kp, des = orb.compute(image, kp)

    return kp, des

In [ ]:
def extract_features_dataset(images, extract_features_function):

    kp_list = []
    des_list = []
    
    for img in images:
        kp , des = extract_features_function(img)
        kp_list.append(kp)
        des_list.append(des)
    
    return kp_list, des_list

In [ ]:
def match_features(des1, des2):

    # # FLANN parameters
    # FLANN_INDEX_KDTREE = 1
    # index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    # search_params = dict(checks=50)   # or pass empty dictionary
    # flann = cv.FlannBasedMatcher(index_params,search_params)
    # matches = flann.knnMatch(des1,des2,k=2)

    # # FLANN ORB
    # FLANN_INDEX_LSH = 6
    # index_params= dict(algorithm = FLANN_INDEX_LSH,table_number = 6,key_size = 12,multi_probe_level = 1)
    # search_params = dict(checks=50)
    # flann = cv.FlannBasedMatcher(index_params,search_params)
    # matches = flann.knnMatch(des1,des2,k=2)


    # BF parameters
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)

    return matches

In [ ]:
def filter_matches_distance(match, dist_threshold):
    filtered_match = []
    for i,(m,n) in enumerate(match):
        if m.distance < dist_threshold*n.distance:
            filtered_match.append(m)

    return filtered_match

In [ ]:
# Change function here to extract feature from SURF/ORB/SIFT

kp_list, des_list = extract_features_dataset(imgs,extract_feature_ORB)
dist_threshold = 0.7
matches = []
for i in range(0,len(des_list)-1):
    match = match_features(des_list[i],des_list[i+1])
    matches.append(filter_matches_distance(match, dist_threshold))

In [ ]:
def show_match(img1,kp1,img2,kp2,matches):
    # img = cv.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    img = cv.drawMatches(img1,kp1,img2,kp2,matches,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    return img

In [ ]:
# Fetching matches
j = 0
matched_img = []
for i in range(0,len(imgs)-1):
    matched_img.append(show_match(imgs[i],kp_list[i],imgs[i+1],kp_list[i+1],matches[j]))
    j += 1


In [ ]:
a1 = 1
for image in matched_img:
    image = cv.resize(image, (1920,1080))
    cv.imwrite("/content/drive/MyDrive/Vision/BF/ORB/Outdoor/match{}.jpg".format(a1), image)
    cv.waitKey(0)
    a1 += 1